In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
raw = pd.read_csv("../input/foursquare-location-matching/train.csv")
raw.head()
rawt = pd.read_csv("../input/foursquare-location-matching/test.csv")
rawt.head()

In [ ]:
test = rawt[['id','latitude','longitude','country','categories']]
test = test.fillna(0)
test.head()

In [ ]:
# train['country'].value_counts()["GA"]

In [ ]:
def distance(lat1,lon1,lat2,lon2):
    from math import sin, cos, sqrt, atan2, radians
    R = 6373.0

    lat1 = radians(abs(lat1))
    lon1 = radians(abs(lon1))
    lat2 = radians(abs(lat2))
    lon2 = radians(abs(lon2))
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
distance(43.451922,-8.329697,40.473145,-3.687787)

In [ ]:
lats = test['latitude'].to_list()
lons = test['longitude'].to_list()
Z = tuple(zip(lats,lons))


In [ ]:
len(test)

In [ ]:
from scipy import spatial
tree = spatial.KDTree(Z)
if len(test)>10:
    loc_data = (tree.query(Z,20))
else:
    loc_data = (tree.query(Z,4))

In [ ]:
loc_data

In [ ]:
list(loc_data[1][0])

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
loc_data

In [ ]:
from tqdm import tqdm
sol = {'id':[],'matches':[]}
for i,r in tqdm(test.iterrows(),total=test.shape[0]):
    indx = loc_data[1][i]
    sol['id'].append(r['id'])
    matched = []
    matched.append(test.iloc[list(indx)[0],:]['id'])
    for j in list(indx)[1:]:
        r2 = test.iloc[j,:]
#         print(str(r['categories']),str(r2['categories']))
        s = similar(str(r['categories']),str(r2['categories']))
        if s>=0.7:
            matched.append(r2['id'])
    sol['matches'].append(' '.join(matched))


In [ ]:
solution = pd.DataFrame(sol)
solution.to_csv('submission.csv',index=False)
solution.head()
